In [2]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('trmodel', binary=True)

In [4]:
# -*- coding: utf-8 -*-
import os
import re
import sys
import string
from os.path import basename
from nltk.corpus import stopwords
from collections import Counter

SAMPLE_FILE_NAME = "all_words.txt"
PY_MODULE_PATH = "cleared_samples"
FILE_NAME = 'all_words.txt'
PATH_OF_SAMPLES = [os.path.join(PY_MODULE_PATH, file) for file in os.listdir(PY_MODULE_PATH)]
ZERO_KEY = "<ZERO>"


def get_txts():
    remov = set(stopwords.words("turkish"))
    topic_dict = {}
    for path in PATH_OF_SAMPLES:
        class_name = basename(path)
        with open(os.path.join(path, FILE_NAME),'r',encoding="utf-8",errors='ignore') as f:
            words = f.readlines()
            words = [w.strip() for w in words if '\n' != w]
            words_list = [w for w in words if w not in remov]
            words_list = [x for x in words_list if not any(c.isdigit() for c in x)]
            words_list = filter (lambda word: len (word) > 2, words_list)
            topic_dict[class_name] = words_list
    return topic_dict

# print("0" in my_dict["dunya"])

def word_dict_f(my_dict):
    all_words = [w for key in my_dict for w in my_dict[key]]
    count_dict = Counter(all_words)
    sorted_dict = {k: v for k, v in sorted(count_dict.items(), key=lambda item: item[1], reverse=True)}
    return sorted_dict
    
my_dict = get_txts()
word_dict = word_dict_f(my_dict)
print("Done")

Done


In [9]:
from nltk.tokenize import word_tokenize
import string

THRESHOLD_TXT_WORD = 20
BASE_PATH = "42bin_haber/news"

PATHS_OF_CLASSES = [os.path.join(BASE_PATH, file) for file in os.listdir(BASE_PATH)]

def get_training_data_w2v(PATHS_OF_CLASSES, word_dict, word_vectors):
    
    vec_mat_y = np.eye(len(PATHS_OF_CLASSES))
    dict_y = {basename(path): vec_mat_y[PATHS_OF_CLASSES.index(path)] for path in PATHS_OF_CLASSES}
    training_data = []
    ##
    folder = 0
    ##
    for path in PATHS_OF_CLASSES:
        class_name = basename(path)
        y_data = dict_y[class_name]
        ##
        fil = 0
        ##
        for file in os.listdir(path):
            fil+=1
            print(class_name,":",fil)
            if file.endswith('.txt'):
                with open(os.path.join(path, file),'r',encoding="utf-8",errors='ignore') as f:
                    text = f.read()
                    word_list = clear_sample(text, word_dict, word_vectors)   ## you can change clear type by writing new function
                    X_data = [word_vectors[w] for w in word_list]
                    training_data.append([X_data, y_data])
    
    return training_data
                    
        
def clear_sample(text, word_dict, word_vectors):
    remov = set(stopwords.words("turkish"))
    words = word_tokenize(text)
    result_words = []
    for w in words:
        temp = w.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).split()
        result_words.extend(t for t in temp)
        
    words = [word.lower() for word in result_words]
    words = [w.strip() for w in words if '\n' or '' != w]
    words = [w for w in words if w not in remov]
    words = [x for x in words if not any(c.isdigit() for c in x)]
    words = filter (lambda word: len (word) > 2, words)
    
    w_list = []
    
    for w in words:
        if (w in word_dict) and (w in word_vectors.vocab):
            w_list.append(w)
        if len(w_list) == THRESHOLD_TXT_WORD:
            break
    
    while len(w_list) < THRESHOLD_TXT_WORD:
        w_list.append(ZERO_KEY)
        
    return w_list

In [10]:
import numpy as np
VEC_SIZE = len(word_vectors["bir"])
zero_vec = np.float32(np.zeros((1,400),dtype=float)[0])
word_vectors.add(ZERO_KEY,zero_vec) 

In [11]:
training_data = get_training_data_w2v(PATHS_OF_CLASSES, word_dict, word_vectors)

 : 622
turkiye : 623
turkiye : 624
turkiye : 625
turkiye : 626
turkiye : 627
turkiye : 628
turkiye : 629
turkiye : 630
turkiye : 631
turkiye : 632
turkiye : 633
turkiye : 634
turkiye : 635
turkiye : 636
turkiye : 637
turkiye : 638
turkiye : 639
turkiye : 640
turkiye : 641
turkiye : 642
turkiye : 643
turkiye : 644
turkiye : 645
turkiye : 646
turkiye : 647
turkiye : 648
turkiye : 649
turkiye : 650
turkiye : 651
turkiye : 652
turkiye : 653
turkiye : 654
turkiye : 655
turkiye : 656
turkiye : 657
turkiye : 658
turkiye : 659
turkiye : 660
turkiye : 661
turkiye : 662
turkiye : 663
turkiye : 664
turkiye : 665
turkiye : 666
turkiye : 667
turkiye : 668
turkiye : 669
turkiye : 670
turkiye : 671
turkiye : 672
turkiye : 673
turkiye : 674
turkiye : 675
turkiye : 676
turkiye : 677
turkiye : 678
turkiye : 679
turkiye : 680
turkiye : 681
turkiye : 682
turkiye : 683
turkiye : 684
turkiye : 685
turkiye : 686
turkiye : 687
turkiye : 688
turkiye : 689
turkiye : 690
turkiye : 691
turkiye : 692
turkiye : 693

In [15]:
np.random.shuffle(training_data)
np.save("training_data_w2v_shuffled.npy", training_data)